（这比赛我才打了没几天，中文选手可以加我微信chixujohnny，可以拉群探讨一下）

# 思路

## 解决一个什么问题？
预测test数据集中，每一个可交易的时间节点，是做【action】操作还是【pass】操作，可以转化为一个是否要做action的一个二分类问题。<br/>
我们预测的是当前这个时间节点，在给定了一坨feature之后，当前这个节点是否要action=1的一个二分类。<br/>
注意不是给定当前feature预测下一个或未来某个时间节点的二分类问题，所以数据加工上会简单一些。<br/>

evaluation原文如下：Each row in the test set represents a trading opportunity for which you will be predicting an action value, 1 to make the trade and 0 to pass on it.
<br/><br/>

## target是什么
我觉得探查数据之前应该先明确target。<br/>
你可能要问了，不就是上面是否要做action的二分类问题吗？<br/>
但实际上原数据集中给的并不是action这样一个label，而根据题意，每一个时间节点是否发生action并能不能产生reward，是由6个数据共同决定的：
* weight
* resp
* resp_1
* resp_2
* resp_3
* resp_4 

公式为：pi=∑j(weightij∗respij∗actionij)<br/>
我发现了一个细节，就是evaluation公式里面，只给了一个respij（也就是在某日期下某时间节点的一个收益）<br/>
这个收益没有明确的表示是哪个resp，因为数据集中给了5种resp<br/>
所以一般这种“赛方故意说的比较含糊”的点，会成为一个头部队伍拉开差距的地方。<br/><br/>
既然我们希望这个pi越大越好，且后面做的还是∑，所以我们要尽可能的保证respij是正数。<br/>
并且简单探查了一下数据发现，weight还有0的，所以【要把这些混淆视听的weight=0的sample全都剃掉】<br/><br/>
说了这么多那target到底是啥？<br/>
我觉得是一个比较开放性的判断，可以依次将每种resp都试一下，when resp_x > 0 then 1 else 0<br/>
可以将output做成5个target，最后对预测的output做个emsemble试试看也行。

## feature怎么加工
### 缺失值填充
看了下，130个特征里只有50个特征是完全没有NaN值的。<br/>
我把所有特征的分布拉了一下，是符合正态分布的，所以用mean填一下就行了。<br/>
（我还有想法用剩余的50个特征拟合一个Regression预测一下剩余feature里的NaN值，notebook GPU时间受限，暂时没时间搞了，感觉会有点用，不过应该不会有啥飞跃）<br/>
### 归一化
XGB的化就不用归一化，这个notebook用的NN，同样看了一下feature分布，都还比较均匀，所以不归一化也行（不排除是赛方提前给我们做了一轮加工？）<br/><br/>

## 用什么model
这个notebook我用的NN，其实我非常喜欢XGB的，有几个原因让我用了NN：<br/>
1、公司电脑还回去了，我自己的笔记本的XGB-GPU环境有问题，用optuna寻找最佳超参数太慢了，没时间折腾了。<br/>
2、notebook GPU时间上限也不算太充裕，不如单机debug来的方便。<br/><br/>
其实这个问题我觉得如果想从10% -> 1%冲击一下model是可以的，时间短的话别把太多精力放在model上，收益不大，折腾半天还容易overfitting<br/><br/>

## 如果给我一个月时间把这个比赛搞好？
* 调试5种resp做target比较花时间，这个应该没啥trick只能暴力试一下，毕竟赛方没给我们这几个值在金融领域的含义是啥。<br/>
* 用Regression把缺失值补一下。<br/>
* 多模型融合：有个想法是把GRU-embedding、NN-embedding concat起来丢给XGB。<br/>
* 做一些进阶的数据预处理，比如做一下分箱平滑一下噪音点等等（其实看分布感觉噪音并不明显，强行分箱不排除有效果变差的可能）<br/>
* 【一些特别玄学的东西很重要】泡在Discussion里多看看别人的思路，比如我看到一个Master说，date=85之前交易频率明显比86~500天要高，反正看大家聊了一通也不知道因为啥，大概率可能是因为交易模型在那个时候发生了改版？原来因为收益不行要较高的频率操作，之后行情好了放慢了频率？等等一些奇怪的猜测...反正我听取的建议，踢掉了前85天的train数据，效果是有提升的，约等于剃掉噪音点，只不过这个噪音是连续的一大坨。<br/>我还看到有人把预测的threshole上调了几个万分点，score又怼上去一点的例子...反正挺玄学的。

---------------------------------------------------------------------------------------------------------------------------------------------------------------------

## TODO LIST（1.26更新）
* 一个离线评估函数，做一下valid调参【doing】
* 开始做特征工程，先弄点滑窗特征看一下吧，从SQL转pandas真的非常不适应
* 再开一个notebook好好做一下EDA
* 有些feature不遵循正态分布，准备单独把这些feature抽出来单独train一下

---------------------------------------------------------------------------------------------------------------------------------------------------------------------

## 一些想法
* 看了另一个大神的NN模型的讨论区，有一个GM说他fit的模型实际上在A榜是overfitting的，他俩差点没打起来，其实我觉得打初赛刷rank就是要不断的overfitting A榜的feature啊，我之前也有A榜冠军开心的一比，B榜overfitting直接变SB的经历。反正我觉得吧，现在这个Leaderboard没啥太大的参考意义，我觉得头部队伍到了B榜基本全部都要完蛋的节奏（当然如果你只是刷A榜型选手那当我没说）
* 【玄学事件】有关随机种子的问题我看讨论区也有很多人讨论，不同的随机种子会造成甚至上千的score差异，如果一个模型fit的相对比较完美是不可能出现这种情况的，非常迷，这块我暂时再观察一下讨论区看看，截止到1.26还没什么想法。
* 这个问题既然是多目标优化问题，那为什么不把weight当成一个target优化呢，我看完全没有人这么做，试一试？

## Loading dataset...

In [ ]:
# coding: utf-8

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import numpy as np
import pandas as pd

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import datatable as dt

#  初始化一个随机数种子
SEED = 1111
np.random.seed(SEED)
tf.random.set_seed(SEED)

print('Loading data...')
train_datatable = dt.fread('/kaggle/input/jane-street-market-prediction/train.csv')  # 用datatable会快一点
train = train_datatable.to_pandas()
del train_datatable
print('Done!')

## Data preprocessing...

In [ ]:
print('Data preprocessing...')
# train = train.query('date > 85').reset_index(drop = True)   # 只保留第86天及以后的data
train = train.query('date > 85').query('ts_id > 13000').reset_index(drop = True)   # 只保留第86天及以后的data
train = train[train['weight'] != 0]

train.fillna(train.mean(),inplace=True)

# train['action'] = ((train['resp'].values) > 0).astype(int)

features = []
for item in train.columns:
    if 'feature' in item:
        features.append(item)

f_mean = np.mean(train[features[1:]].values,axis=0)  # 算一下每个feature的均值，用于缺失值填充

print('Done!')

## Make X and y...

In [ ]:
print('Make X and y...')

X_train = train.loc[:, train.columns.str.contains('feature')]

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T

del train

print('Done!')

## Define a easy NN model...

In [ ]:
print('Create a easy NN model...')

HIDDEN_LAYER = [150, 150, 150]
TARGET_NUM = 5   # 优化那5个resp

input = tf.keras.layers.Input(shape=(len(features), ))

x = tf.keras.layers.BatchNormalization()(input)
x = tf.keras.layers.Dropout(0.2)(x)
for units in HIDDEN_LAYER:
    x = tf.keras.layers.Dense(units)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)  # 除了ReLU还可以试试别的，后面可以做一个多模型融合
    x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(TARGET_NUM)(x)

output = tf.keras.layers.Activation("sigmoid")(x)

model = tf.keras.models.Model(inputs=input, outputs=output)
model.compile(
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3),
    metrics   = tf.keras.metrics.AUC(name="AUC"),
    loss      = tf.keras.losses.BinaryCrossentropy(label_smoothing=1e-2),
)

print('Done!')

## Train NN...

In [ ]:
print('Train NN...')

history = model.fit(X_train, y_train, epochs=5, batch_size=3000)
models = []
models.append(model)

print('Done!')

## See what is the Learning Curve looks like...

Accroding to this learning curve.<br/>
Maybe 18 epoch is a better chioce? <br/>
But it seems to be in a state of continuous overfitting.<br/>
Change prams should be a way to increace score.<br/>

In [ ]:
print('See what is the Learning Curve looks like...')
import matplotlib.pyplot as plt

# 绘制训练 & 验证的准确率值
plt.plot(history.history['AUC'])
# plt.plot(history.history['val_AUC'])
plt.title('Model accuracy')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# 绘制训练 & 验证的损失值
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Free the Memory...

In [ ]:
del X_train, y_train

# 自己添加代码1.28


In [ ]:
testData = pd.read_csv('../input/jane-street-market-prediction/example_test.csv')
testData = testData.iloc[:,1:-2]

# 通过模型预测出来的label
predict = model.predict(testData)

# 将数组predict 转成 dataFrame
DFpredict = pd.DataFrame(predict)
# print(DFpredict)

# 概览predict
# print(predict)

# 查看训练集的列名
# print(X_train.columns.values)

# reshape测试集的feature
# print(testData.iloc[:,1:-2].columns.values)

# 打印预测集的label shape
# print(np.shape(predict))

# 打印训练集的label shape
# print(np.shape(y_train))

# print(testData)
# print(np.size(predict))

# 合并预测结果和example_sample_submission
submit = pd.read_csv('../input/jane-street-market-prediction/example_sample_submission.csv')
# print(submit)
mergeResult = pd.concat([submit, DFpredict],axis=1)
# print(mergeResult)
# print(submit)

# 填充Nan
mergeResult.fillna(mergeResult.mean(),inplace=True)
print(mergeResult)
# 保存预测的结果
mergeResult.to_csv('./submission.csv')

print('done')



## Predict...

In [ ]:
# THRESHOLD = 0.5   # 试了一个对照组，还是0.5比较好用

# import janestreet
# from tqdm import tqdm
# janestreet.make_env.__called__ = False
# env = janestreet.make_env()
# for (test_df, pred_df) in tqdm(env.iter_test()):
#     if test_df['weight'].item() > 0:
#         test_samples = test_df.loc[:, features].values  # 遍历每个样本的feature部分，做成一个一行130列的matrix
#         if np.isnan(test_samples[:, 1:].sum()):         # 如果这行样本里面有NaN的话（忽略离散特征feature_0）用train数据集里的mean替换NaN
#             test_samples[:, 1:] = np.nan_to_num(test_samples[:, 1:]) + np.isnan(test_samples[:, 1:]) * f_mean  
#         pred = model(test_samples, training = False).numpy()
        
#         # version = 1
#         pred = np.median(pred)
        
#         # version = 2
# #         pred = np.mean(pred)
        
#         # version = 3
# #         pred = pred[3]
        
#         pred_df.action = np.where(pred >= THRESHOLD, 1, 0).astype(int)
#     else:
#         pred_df.action = 0
#     env.predict(pred_df)